# taulab - Usage Examples

This notebook demonstrates the main features of the taulab package for physics lab data analysis.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

## 1. Data Types

taulab provides several data types for working with physics measurements.

In [ ]:
from taulab.datatypes import PhysicalSize, ParseResult, FitResult

# PhysicalSize: Represents a measured value with uncertainty
mass = PhysicalSize(value=5.67, uncertainty=0.03)
print(f"Mass: {mass.value} ± {mass.uncertainty} kg")

length = PhysicalSize(10.2, 0.1)
print(f"Length: {length.value} ± {length.uncertainty} m")

## 2. Statistical Analysis

The `nsigma` function calculates how many standard deviations apart two measurements are. This is useful for determining if two values agree within uncertainty.

In [ ]:
from taulab.stats import nsigma

# Compare two measurements to check if they agree within uncertainty
measured_g = PhysicalSize(9.78, 0.05)
expected_g = PhysicalSize(9.81, 0.01)

sigma_diff = nsigma(measured_g, expected_g)
print(f"Measured g differs from expected by {sigma_diff:.2f} sigma")

## 3. Curve Fitting with ODR

ODR (Orthogonal Distance Regression) accounts for uncertainties in both x and y values, making it ideal for physics experiments.

In [ ]:
from taulab.fit import odr_fit, fit_functions

# Generate synthetic data: y = 2x + 3 with noise
np.random.seed(42)
x_data = np.linspace(0, 10, 20)
x_err = np.full_like(x_data, 0.1)  # 0.1 uncertainty in x
y_true = 2 * x_data + 3
y_data = y_true + np.random.normal(0, 0.5, len(x_data))
y_err = np.full_like(y_data, 0.5)  # 0.5 uncertainty in y

# Quick visualization of the data
plt.figure(figsize=(8, 5))
plt.errorbar(x_data, y_data, xerr=x_err, yerr=y_err, fmt='o', label='Data')
plt.plot(x_data, y_true, 'r--', label='True: y = 2x + 3')
plt.xlabel('x')
plt.ylabel('y')
plt.legend()
plt.title('Synthetic Data for Fitting')
plt.show()

In [ ]:
# Fit using built-in linear function: f(A, x) = A[1]*x + A[0]
initial_guess = [1.0, 1.0]  # [intercept, slope]

fit_result = odr_fit(
    fit_func=fit_functions.linear,
    init_values=initial_guess,
    x_data=x_data,
    x_err=x_err,
    y_data=y_data,
    y_err=y_err,
)

print("Linear Fit Results")
print("=" * 40)
print(f"Intercept (A[0]): {fit_result.params[0]:.3f} ± {fit_result.error[0]:.3f}")
print(f"Slope (A[1]):     {fit_result.params[1]:.3f} ± {fit_result.error[1]:.3f}")
print(f"\nTrue values:      intercept=3, slope=2")

In [ ]:
# Use extrapolate to predict values at new x positions
x_new = np.array([15, 20])
y_predicted = fit_result.extrapolate(x_new)

print("Extrapolation:")
print(f"Predicted y at x=15: {y_predicted[0]:.2f}")
print(f"Predicted y at x=20: {y_predicted[1]:.2f}")

## 4. Available Fit Functions

taulab provides several pre-built functions for common physics relationships.

In [ ]:
print("Available Fit Functions")
print("=" * 50)
print()
print("linear(A, x)      = A[1]*x + A[0]")
print("polynomial(A, x)  = A[2]*x² + A[1]*x + A[0]")
print("polynomial_n(n)   = Creates n-th order polynomial")
print("exponential(A, x) = A[2]*exp(A[1]*x) + A[0]")
print("sinusoidal(A, x)  = A[3]*sin(A[1]*x + A[2]) + A[0]")
print("logarithmic(A, x) = A[2]*log(A[1]*x) + A[0]")
print("optics(A, x)      = A[1]*x/(x - A[1]) + A[0]")

In [ ]:
# Example: Create a cubic polynomial using the factory function
cubic = fit_functions.polynomial_n(3)

# This creates: A[3]*x³ + A[2]*x² + A[1]*x + A[0]
x_test = np.array([1, 2, 3])
A_test = [1, 2, 3, 4]  # coefficients
print(f"Cubic at x={x_test}: {cubic(A_test, x_test)}")

## 5. Custom Fit Functions

You can define your own fit functions following the convention: `f(A, x)` where `A` is the parameter array and `x` is the independent variable.

In [ ]:
# Define a Gaussian function
def gaussian(A, x):
    """Gaussian: A[0] + A[1] * exp(-((x - A[2])² / (2 * A[3]²)))"""
    return A[0] + A[1] * np.exp(-((x - A[2]) ** 2) / (2 * A[3] ** 2))

# Generate Gaussian data
x_gauss = np.linspace(-5, 5, 50)
x_gauss_err = np.full_like(x_gauss, 0.05)
y_gauss_true = gaussian([0, 10, 0, 1], x_gauss)
y_gauss = y_gauss_true + np.random.normal(0, 0.3, len(x_gauss))
y_gauss_err = np.full_like(y_gauss, 0.3)

# Visualize
plt.figure(figsize=(8, 5))
plt.errorbar(x_gauss, y_gauss, xerr=x_gauss_err, yerr=y_gauss_err, fmt='o', alpha=0.5, label='Data')
plt.plot(x_gauss, y_gauss_true, 'r-', label='True Gaussian')
plt.xlabel('x')
plt.ylabel('y')
plt.legend()
plt.title('Gaussian Data')
plt.show()

In [ ]:
# Fit with custom Gaussian function
gauss_fit = odr_fit(
    fit_func=gaussian,
    init_values=[0, 8, 0.5, 1.5],  # [baseline, amplitude, center, width]
    x_data=x_gauss,
    x_err=x_gauss_err,
    y_data=y_gauss,
    y_err=y_gauss_err,
)

print("Custom Gaussian Fit Results")
print("=" * 40)
print(f"Baseline (A[0]):  {gauss_fit.params[0]:.3f} ± {gauss_fit.error[0]:.3f}")
print(f"Amplitude (A[1]): {gauss_fit.params[1]:.3f} ± {gauss_fit.error[1]:.3f}")
print(f"Center (A[2]):    {gauss_fit.params[2]:.3f} ± {gauss_fit.error[2]:.3f}")
print(f"Width σ (A[3]):   {gauss_fit.params[3]:.3f} ± {gauss_fit.error[3]:.3f}")
print(f"\nTrue values:      baseline=0, amplitude=10, center=0, width=1")

## 6. Graphing

The `Graph` class provides convenient plotting of data with fit curves and residual analysis.

In [ ]:
from taulab.graph import Graph

# Create a Graph object from fit result and data
graph = Graph(
    fit_result=fit_result,
    x_data=x_data,
    y_data=y_data,
    x_err=x_err,
    y_err=y_err,
)

In [ ]:
# Create figure with two subplots: data+fit and residuals
fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(8, 8), height_ratios=[2, 1])

# Plot data with fit
graph.plot(
    ax=ax1,
    data_label="Experimental data",
    fit_label="Linear fit",
)
ax1.set_xlabel("x")
ax1.set_ylabel("y")
ax1.set_title("Linear Fit Example")

# Plot residuals
graph.residuals_plot(
    ax=ax2,
    residuals_label="Residuals",
)
ax2.set_xlabel("x")
ax2.set_ylabel("y - fit")

plt.tight_layout()
plt.show()

In [ ]:
# You can also extrapolate the fit over a custom x range
fig, ax = plt.subplots(figsize=(8, 5))

x_extended = np.linspace(-2, 15, 100)
graph.plot(
    x_fit=x_extended,
    ax=ax,
    data_label="Data",
    fit_label="Extrapolated fit",
)
ax.set_xlabel("x")
ax.set_ylabel("y")
ax.set_title("Fit with Extrapolation")
ax.axvline(x=0, color='gray', linestyle=':', alpha=0.5)
ax.axvline(x=10, color='gray', linestyle=':', alpha=0.5, label='Data range')
plt.show()

## 7. Parsing Data Files

taulab can batch-load data files from a folder, optionally extracting metadata from filenames using regex patterns.

In [ ]:
from taulab.parse import parse_excel_folder, parse_csv_folder

# TODO: Add example here